In [1]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("churn-data-3333.csv")

In [2]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()
num_df.head()

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,128,415,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


In [27]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)
cat_df=cat_df.drop(['phone number'], axis=1)



#BOX-COX
#lemda=0.5
#num_df=(num_df**lemda)
#num_df=num_df-1
#num_df[num_df < 0]=0
#num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
#num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
#num_df=round(np.log(num_df.add(1)),2)
#num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log

num_df=num_df.fillna(conData)


In [28]:
for col in num_df.columns:
    Bins=20
    Omega=num_df[col].max()-num_df[col].min()
    #Omega=num_df[col].max()-0
    Omega=Omega/Bins
    #print(num_df[col].min())
    #BINValue=[1.1, 1.1]
    BINValue=[]
    BINValue.insert(0,num_df[col].min())
    #BINValue.insert(0,-1)
    for i in range(19):
        BINValue.insert(i+1,BINValue[i]+Omega)
        #print(BINValue[i+1])
    #print(BINValue)
    #print(num_df[col])
    labels =[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    cats=pd.cut(num_df[col], BINValue, labels=labels)
    num_df[col]=cats
    #print(cats)
    #cats.to_csv('D://cats.csv')
    #break

In [29]:
num_df=num_df.fillna(1)
result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

#result_df=result_df.replace([np.inf, -np.inf], np.nan)
#result_df=result_df.fillna(-1)
#result_df=result_df.abs()

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn'] 
y=churn

In [30]:
X.shape

(3333, 19)

In [31]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
15,number_customer_service_calls,253.822022
17,international_plan,203.244178
2,number_vmail_messages,167.085298
5,total_day_charge,94.226803
3,total_day_minutes,94.047195
18,voice_mail_plan,25.156959
13,total_intl_calls,15.885968
6,total_eve_minutes,13.090907
8,total_eve_charge,13.013255
12,total_intl_minutes,9.186276


In [32]:
y_train=y
x_train_chi = select_feature.transform(X_train)
x_train_chi.shape

(3333, 15)

In [33]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

2622 228 222 261
pod:  0.5403726708074534
pof:  0.08
AUC:  0.7301863354037267


In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.7s


{'var_smoothing': 0.0005336699231206307}
GaussianNB(priors=None, var_smoothing=0.0005336699231206307)
0.8832883288328833


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:   16.4s finished


26710 23728 21590 27972
pod:  0.5643840038739357
pof:  0.4704389547563345
AUC:  0.5469725245588006


In [35]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=0.0005336699231206307)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

2741 109 281 202
pod:  0.41821946169772256
pof:  0.03824561403508772
AUC:  0.6899869238313174
accuracy:  0.882988298829883


In [1]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 200 tasks      | elapsed:   18.3s


{'C': 2, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
0.8592859285928592


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   20.1s finished


In [37]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2786 64 405 78
pod:  0.16149068322981366
pof:  0.02245614035087719
AUC:  0.5695172714394683
accuracy:  0.8592859285928592


In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'uniform'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
0.8811881188118812


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    1.6s finished


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear'], 
                     'gamma': [0.001, 0.01, 0.1, 1, 1e-3, 1e-4], 
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} ]

grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 410.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 1993.6min


In [39]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform' )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


2820 30 362 121
pod:  0.2505175983436853
pof:  0.010526315789473684
AUC:  0.6199956412771058
accuracy:  0.8823882388238824


In [40]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


2831 19 386 97
pod:  0.20082815734989648
pof:  0.006666666666666667
AUC:  0.5970807453416149
accuracy:  0.8784878487848785


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   55.9s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  2.4min finished


{'bootstrap': True, 'max_depth': 100, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.946894689468947


In [10]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

{'bootstrap': True, 'max_depth': 110, 'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=110, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.56987


In [42]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(
bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2825 25 154 329
pod:  0.6811594202898551
pof:  0.008771929824561403
AUC:  0.8361937452326468
accuracy:  0.9462946294629463


In [43]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2850 0 480 3
pod:  0.006211180124223602
pof:  0.0
AUC:  0.5031055900621118


In [45]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2699 151 124 359
pod:  0.7432712215320911
pof:  0.052982456140350874
AUC:  0.8451443826958701
accuracy:  0.9174917491749175


In [49]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2675 175 133 350
pod:  0.7246376811594203
pof:  0.06140350877192982
AUC:  0.8316170861937453


In [51]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2807 43 137 346
pod:  0.7163561076604554
pof:  0.015087719298245613
AUC:  0.8506341941811049
accuracy:  0.945994599459946


In [50]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2806 44 137 346
pod:  0.7163561076604554
pof:  0.015438596491228071
AUC:  0.8504587555846137
accuracy:  0.9456945694569457


In [52]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 


Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 5s 2ms/step - loss: 0.4345 - acc: 0.8579 - val_loss: 0.4805 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 151us/step - loss: 0.4170 - acc: 0.8587 - val_loss: 0.4360 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 150us/step - loss: 0.4101 - acc: 0.8587 - val_loss: 0.4217 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 136us/step - loss: 0.3984 - acc: 0.8587 - val_loss: 0.4169 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 143us/step - loss: 0.3955 - acc: 0.8587 - val_loss: 0.4082 - val_acc: 0.8417
Epoch 6/30
2399/2399 [==============================] - 0s 148us/step - loss: 0.3877 - acc: 0.8587 - val_loss: 0.4050 - val_acc: 0.8417
Epoch 7/30
2399/2399 [==============================] - 0s 139us/step - loss: 0.3818 - acc: 0.8583 - val_loss: 0.3993 - val_acc: 0.8450
Epo

Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 5s 2ms/step - loss: 0.4981 - acc: 0.8495 - val_loss: 0.4732 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 196us/step - loss: 0.4116 - acc: 0.8587 - val_loss: 0.4273 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 196us/step - loss: 0.3948 - acc: 0.8587 - val_loss: 0.4088 - val_acc: 0.8433
Epoch 4/30
2399/2399 [==============================] - 0s 168us/step - loss: 0.3844 - acc: 0.8649 - val_loss: 0.3879 - val_acc: 0.8433
Epoch 5/30
2399/2399 [==============================] - 0s 146us/step - loss: 0.3691 - acc: 0.8666 - val_loss: 0.3853 - val_acc: 0.8433
Epoch 6/30
2399/2399 [==============================] - 0s 139us/step - loss: 0.3622 - acc: 0.8712 - val_loss: 0.3871 - val_acc: 0.8533
Epoch 7/30
2399/2399 [==============================] - 0s 140us/step - loss: 0.3746 - acc: 0.8624 - val_loss: 0.3495 - val_acc: 0.8600
Epo

Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 4s 2ms/step - loss: 0.4268 - acc: 0.8474 - val_loss: 0.4088 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 163us/step - loss: 0.4032 - acc: 0.8587 - val_loss: 0.4017 - val_acc: 0.8400
Epoch 3/30
2399/2399 [==============================] - 0s 158us/step - loss: 0.3938 - acc: 0.8566 - val_loss: 0.3977 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 134us/step - loss: 0.3894 - acc: 0.8562 - val_loss: 0.4234 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 143us/step - loss: 0.3857 - acc: 0.8574 - val_loss: 0.3953 - val_acc: 0.8467
Epoch 6/30
2399/2399 [==============================] - 0s 143us/step - loss: 0.3773 - acc: 0.8604 - val_loss: 0.3889 - val_acc: 0.8500
Epoch 7/30
2399/2399 [==============================] - 0s 135us/step - loss: 0.3776 - acc: 0.8654 - val_loss: 0.3746 - val_acc: 0.8533
Epo

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 4s 2ms/step - loss: 0.4446 - acc: 0.8454 - val_loss: 0.4804 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 153us/step - loss: 0.4099 - acc: 0.8583 - val_loss: 0.4091 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 153us/step - loss: 0.4085 - acc: 0.8583 - val_loss: 0.4063 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 151us/step - loss: 0.3986 - acc: 0.8600 - val_loss: 0.4002 - val_acc: 0.8433
Epoch 5/30
2400/2400 [==============================] - 0s 146us/step - loss: 0.3930 - acc: 0.8588 - val_loss: 0.3880 - val_acc: 0.8500
Epoch 6/30
2400/2400 [==============================] - 0s 148us/step - loss: 0.3884 - acc: 0.8579 - val_loss: 0.3866 - val_acc: 0.8500
Epoch 7/30
2400/2400 [==============================] - 0s 151us/step - loss: 0.3816 - acc: 0.8596 - val_loss: 0.3903 - val_acc: 0.8450
Epo

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 4s 2ms/step - loss: 0.4307 - acc: 0.8583 - val_loss: 0.4285 - val_acc: 0.8467
Epoch 2/30
2400/2400 [==============================] - 0s 132us/step - loss: 0.3981 - acc: 0.8588 - val_loss: 0.4078 - val_acc: 0.8400
Epoch 3/30
2400/2400 [==============================] - 0s 143us/step - loss: 0.3897 - acc: 0.8625 - val_loss: 0.4006 - val_acc: 0.8517
Epoch 4/30
2400/2400 [==============================] - 0s 148us/step - loss: 0.3866 - acc: 0.8621 - val_loss: 0.3953 - val_acc: 0.8517
Epoch 5/30
2400/2400 [==============================] - 0s 140us/step - loss: 0.3768 - acc: 0.8621 - val_loss: 0.3879 - val_acc: 0.8500
Epoch 6/30
2400/2400 [==============================] - 0s 143us/step - loss: 0.3699 - acc: 0.8621 - val_loss: 0.3861 - val_acc: 0.8433
Epoch 7/30
2400/2400 [==============================] - 0s 146us/step - loss: 0.3639 - acc: 0.8662 - val_loss: 0.3827 - val_acc: 0.8517
Epo

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 4s 2ms/step - loss: 0.4643 - acc: 0.8483 - val_loss: 0.4346 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 159us/step - loss: 0.4160 - acc: 0.8583 - val_loss: 0.4524 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 148us/step - loss: 0.4064 - acc: 0.8583 - val_loss: 0.4256 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 148us/step - loss: 0.4046 - acc: 0.8583 - val_loss: 0.4052 - val_acc: 0.8433
Epoch 5/30
2400/2400 [==============================] - 0s 155us/step - loss: 0.4029 - acc: 0.8563 - val_loss: 0.4000 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 149us/step - loss: 0.3869 - acc: 0.8588 - val_loss: 0.4222 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 151us/step - loss: 0.3857 - acc: 0.8592 - val_loss: 0.3886 - val_acc: 0.8500
Epo

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 4s 2ms/step - loss: 0.4369 - acc: 0.8467 - val_loss: 0.4246 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 157us/step - loss: 0.4054 - acc: 0.8583 - val_loss: 0.4119 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 146us/step - loss: 0.3970 - acc: 0.8583 - val_loss: 0.4123 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 148us/step - loss: 0.3949 - acc: 0.8583 - val_loss: 0.4089 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 155us/step - loss: 0.3947 - acc: 0.8579 - val_loss: 0.4045 - val_acc: 0.8450
Epoch 6/30
2400/2400 [==============================] - 0s 147us/step - loss: 0.3831 - acc: 0.8592 - val_loss: 0.3921 - val_acc: 0.8483
Epoch 7/30
2400/2400 [==============================] - 0s 149us/step - loss: 0.3774 - acc: 0.8608 - val_loss: 0.3894 - val_acc: 0.8500
Epo

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 4s 2ms/step - loss: 0.4468 - acc: 0.8538 - val_loss: 0.4316 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 155us/step - loss: 0.4083 - acc: 0.8583 - val_loss: 0.4298 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 157us/step - loss: 0.4072 - acc: 0.8583 - val_loss: 0.4178 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 163us/step - loss: 0.4004 - acc: 0.8583 - val_loss: 0.4034 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 173us/step - loss: 0.3942 - acc: 0.8588 - val_loss: 0.4065 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 164us/step - loss: 0.3879 - acc: 0.8575 - val_loss: 0.4057 - val_acc: 0.8433
Epoch 7/30
2400/2400 [==============================] - 0s 156us/step - loss: 0.3842 - acc: 0.8596 - val_loss: 0.4048 - val_acc: 0.8433
y2_

2400/2400 [==============================] - 0s 176us/step - loss: 0.3901 - acc: 0.8629 - val_loss: 0.4345 - val_acc: 0.8233
Epoch 5/30
2400/2400 [==============================] - 0s 168us/step - loss: 0.3845 - acc: 0.8629 - val_loss: 0.4251 - val_acc: 0.8233
Epoch 6/30
2400/2400 [==============================] - 0s 173us/step - loss: 0.3794 - acc: 0.8633 - val_loss: 0.4136 - val_acc: 0.8233
Epoch 7/30
2400/2400 [==============================] - 0s 168us/step - loss: 0.3720 - acc: 0.8625 - val_loss: 0.4423 - val_acc: 0.8233
Epoch 8/30
2400/2400 [==============================] - 0s 167us/step - loss: 0.3700 - acc: 0.8633 - val_loss: 0.4128 - val_acc: 0.8233
Epoch 9/30
2400/2400 [==============================] - 0s 171us/step - loss: 0.3680 - acc: 0.8621 - val_loss: 0.3984 - val_acc: 0.8233
Epoch 10/30
2400/2400 [==============================] - 0s 168us/step - loss: 0.3655 - acc: 0.8633 - val_loss: 0.4104 - val_acc: 0.8217
Epoch 11/30
2400/2400 [==============================] - 0

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 5s 2ms/step - loss: 0.4738 - acc: 0.8521 - val_loss: 0.4557 - val_acc: 0.8233
Epoch 2/30
2400/2400 [==============================] - 0s 177us/step - loss: 0.3970 - acc: 0.8629 - val_loss: 0.4746 - val_acc: 0.8233
Epoch 3/30
2400/2400 [==============================] - 0s 171us/step - loss: 0.3841 - acc: 0.8617 - val_loss: 0.4447 - val_acc: 0.8233
Epoch 4/30
2400/2400 [==============================] - 0s 173us/step - loss: 0.3784 - acc: 0.8625 - val_loss: 0.4300 - val_acc: 0.8283
Epoch 5/30
2400/2400 [==============================] - 0s 177us/step - loss: 0.3775 - acc: 0.8633 - val_loss: 0.4250 - val_acc: 0.8217
Epoch 6/30
2400/2400 [==============================] - 0s 171us/step - loss: 0.3662 - acc: 0.8683 - val_loss: 0.4158 - val_acc: 0.8283
Epoch 7/30
2400/2400 [==============================] - 0s 178us/step - loss: 0.3633 - acc: 0.8658 - val_loss: 0.4415 - val_acc: 0.8267
Epo

In [53]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

Train on 2399 samples, validate on 600 samples
Epoch 1/5
2399/2399 [==============================] - 6s 2ms/step - loss: 0.5299 - acc: 0.8491 - val_loss: 0.4329 - val_acc: 0.8417
Epoch 2/5
2399/2399 [==============================] - 0s 141us/step - loss: 0.3984 - acc: 0.8587 - val_loss: 0.4222 - val_acc: 0.8417
Epoch 3/5
2399/2399 [==============================] - 0s 141us/step - loss: 0.3872 - acc: 0.8587 - val_loss: 0.4389 - val_acc: 0.8417
Epoch 4/5
2399/2399 [==============================] - 0s 139us/step - loss: 0.3803 - acc: 0.8587 - val_loss: 0.4175 - val_acc: 0.8417
Epoch 5/5
2399/2399 [==============================] - 0s 143us/step - loss: 0.3712 - acc: 0.8587 - val_loss: 0.3960 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 6s 2ms/step - loss: 0.5335 - acc: 0.8258 - val_loss: 0.4285 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 130us/step - loss: 0.3949 - acc: 0.8583 - val_loss: 0.4188 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 129us/step - loss: 0.3865 - acc: 0.8583 - val_loss: 0.4242 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 132us/step - loss: 0.3785 - acc: 0.8583 - val_loss: 0.4034 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 131us/step - loss: 0.3696 - acc: 0.8583 - val_loss: 0.4048 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 6s 3ms/step - loss: 0.5339 - acc: 0.8575 - val_loss: 0.4317 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 145us/step - loss: 0.4016 - acc: 0.8583 - val_loss: 0.4245 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 142us/step - loss: 0.3913 - acc: 0.8583 - val_loss: 0.4146 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 146us/step - loss: 0.3834 - acc: 0.8583 - val_loss: 0.4089 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 149us/step - loss: 0.3771 - acc: 0.8583 - val_loss: 0.4078 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 7s 3ms/step - loss: 0.5323 - acc: 0.8458 - val_loss: 0.4714 - val_acc: 0.8233
Epoch 2/5
2400/2400 [==============================] - 0s 145us/step - loss: 0.3925 - acc: 0.8629 - val_loss: 0.4548 - val_acc: 0.8233
Epoch 3/5
2400/2400 [==============================] - 0s 146us/step - loss: 0.3819 - acc: 0.8629 - val_loss: 0.4705 - val_acc: 0.8233
Epoch 4/5
2400/2400 [==============================] - 0s 141us/step - loss: 0.3773 - acc: 0.8629 - val_loss: 0.4325 - val_acc: 0.8233
Epoch 5/5
2400/2400 [==============================] - 0s 150us/step - loss: 0.3701 - acc: 0.8629 - val_loss: 0.4294 - val_acc: 0.8233
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]